<a id="top"></a>
# Topics Outline

## [Why are 2 interacting RR robots different than a 5barlinkage?](#5bl_explanation)
## [What does the most general 5 bar linkage look like?](#g5bl_model)
 - derivation
 - gen5barlinkage class definition
 - gen5barlinkage exploration tool
 - discussion points
 
## [What does the human model look like?](#human_model)
- derivation
- human class definition
- human model exploration tool. 
- discussion points

## [Joint exploration - put the two together](#joint_exp)
- joint exploration tool 

## [Solve the constrained optimization problem](#solution)

# [notes](#notes)


<a id="5bl_explanation"></a>
# Why are 2 interacting RR robots different than a 5-bar-linkage? [ &#x21ea;](#top)



# Utility class

In [79]:

class utils:
    """
    utility functions shared between different classes
    """
    @staticmethod
    def dist(x1,y1,x2,y2):
        """
        calculate the distance between 2 points
        """
        return ((x2 - x1)**2 + (y2 - y1)**2)**.5
    
    @staticmethod
    def GD(x0,df,params):
        """
        peform gradient decent, given df, a function for calculating the 
        derivate of the function to be optimized. 
        """
        next_x = x0   # We start at x0
        gamma = 0.01  # Step size multiplier
        precision = 0.0001  # Desired precision of result
        max_iters = 10000  # Maximum number of iterations
        
        for _ in range(max_iters):
            current_x = next_x
            next_x = current_x - gamma * df(current_x,params)

            step = next_x - current_x
            if abs(step) <= precision:
                break

        return next_x


<a id="g5bl_model"></a>
# What does the most general 5 bar linkage look like? [ &#x21ea;](#top)  

###  below we see a parametric representation of the kinematics of a general 5 bar linkage:


<img src="./imgs/general5bl.svg" style="width:1200px">

### * if we allow $L_{0} = 0$ and attach the end effector to $L_{2}$ with a decent sized $P_{x}$ extension segment, and place $(y_1,x_1)$ at the hip joint and $e_1$ at the knee, then we have the "exoskeleton" like design


In [177]:
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact , Layout , FloatSlider , Checkbox , Dropdown
from IPython.display import display
from numpy import linalg as LA
from shapely.geometry import Polygon

class gen5barlinkage:
    def __init__(self):
        #-------design variables---------
        #continuous state variables
        self.x1 = 0
        self.y1 = 0
        self.x2 = 1
        self.y2 = 1
        
        self.L0 = 1
        self.L1 = 1
        self.L2 = 1
        self.L3 = 1
        self.L4 = 1
        
        self.Px = 0
        self.Py = 0
        
        #discrete state variables
        self.eStates = [0,0,0]  #[only 1 at once]
        self.attachmentPt = 2   #[2 or 3]
        
        #------- internal state vars--------
        self.θ1 = None
        self.θ2 = None
    
    def IK(self,x,y):
        """
        given a location x,y of the midfoot, what are the joint angles required to
        achieve that position? 
        """
        
        
        

<a id="human_model"></a>
# what does the human model look like? [ &#x21ea;](#top)

<img src="./imgs/humanModel.svg" style="width:1200px">


### Discussion Points:
### IK - approaches to making IK realistic
* define springs and minimize a potential energy function
* develop a huristic for $ϕ(x,y)$

<!-- $$
\begin{align}
τ_{h} &= f(θ_{h},θ_{k}) \\
τ_{k} &= f(θ_{h},θ_{k},θ_{a}) \\
τ_{a} &= f(θ_{k},θ_{a})
\end{align}
$$ -->

In [8]:
import plotly.graph_objs as go
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact , Layout , FloatSlider , Checkbox , Dropdown
from IPython.display import display
from numpy import linalg as LA
from shapely.geometry import Polygon
from scipy.spatial import ConvexHull
from scipy.optimize import minimize

    
##design decision - should I treat Human more functionally - so basically as a static class, but without state, and more of a container for functions, or should I rely heavily 
#on state caching, which means I have to guarentee order of function calls inorder for things to be up to date. this is the fundemental tradeoff of functional vs stateful approaches. 
# other design decisions, would sampling happen in cartesian, or in joint space for the optimization? - at least some of that sampling should live in another class that handles the
#optimization between the human aand the 

class JointSpring:
    def __init__(self,θo,K,θmin,θmax):
        self.θo = θo;
        self.K = K;
        self.θmin = θmin
        self.θmax = θmax
        
    def PE(self,θ,mode = "linear"):
        """
        calculates and return the potential energy of the 
        spring given the angular displacement θ and the 
        """
        if mode == "linear":
            return .5 * self.K * (θ - self.θo)**2
        elif mode == "quadratic":
            return (1/3) * self.K *(θ - self.θo)**3
        elif mode == "softLimits":
            pass
        else:
            raise ValueError("mode {} not yet implemented".format(mode))


class Human:
    def __init__(self):
        #segment lengths
        self.femur = .4075    #[meters - from anthopometric tables] 
        self.shank = .390     #[meters] 
        self.foot = .255      #[meters]
        self.reach = self.femur + self.shank + self.foot/2
        
        #joint limits
        self.θh_max = np.pi/180 *  90
        self.θh_min = np.pi/180 * -45
        self.θk_max = np.pi/180 *  0
        self.θk_min = np.pi/180 * -120
        self.θa_max = np.pi/180 * 90
        self.θa_min = np.pi/180 * 10
        
        #IK joint springs
        self.θhSpring = JointSpring(0,.001,self.θh_min,self.θh_max)
        self.θkSpring = JointSpring(0,.5,self.θk_min,self.θk_max)
        self.θaSpring = JointSpring(-np.pi/2,2,self.θa_min,self.θa_max)
        
        #state:
        self.θh = 0
        self.θk = 0
        self.θa = 0 
        

        
    #------------- private functions --------------------    
    def _IK_analytical(self,x_e,y_e,ϕ):
        """
        given a midfoot location and an orientation, return the joint angles
        by solving the analytical IK with wrist - end effector separation
        based on lect5 kinematics - slides 9-10
        """
        #define link lengths
        l1=self.femur
        l2=self.shank
        l3=self.foot/2
        
        #solve first for the wrist location
        x_w = x_e - l3*np.cos(ϕ)
        y_w = y_e - l3*np.sin(ϕ)
        
        #solve for intermediate values
        r = utils.dist(0,0,x_w,y_w)
        β = np.arccos((l1**2 + l2**2 - r**2) / (2*l1*l2))
        γ = np.arccos((r**2 + l1**2 - l2**2) / (2*r*l1))
        α = np.arctan2(y_w,x_w)
        
        #calculate θ's 
        θ1 = α - γ
        θ2 = np.pi - β
        θ3 = ϕ - θ1 - θ2
        
        #calc θ_primes for elbow up config
        θ1pm = θ1 + 2*γ  
        θ2pm = -θ2                      
        θ3pm = θ3 + 2*(θ2 - γ)
        
        return θ1pm,θ2pm,θ3pm
    
    
    def _IK_energy_fxn(self,θh,θk,θa):
        """
        defines the potential energy landscape
        """
        pe_h = self.θhSpring.PE(θh,mode = "linear")
        pe_k = self.θaSpring.PE(θk,mode = "linear")
        pe_a = self.θkSpring.PE(θa,mode = "linear")
        
        return pe_h + pe_k + pe_a
    
    def _energy_grad(self,ϕ,params):
        """
        calculate the gradient of the energy function using 
        finite differences for use in gradient decent
        """
        x = params[0] ; y = params[1]
        δ = .0001
        δm_θh,δm_θhk,δm_θa = self._IK_analytical(x,y,ϕ - δ)
        δp_θh,δp_θhk,δp_θa = self._IK_analytical(x,y,ϕ + δ)
        
        δmPE = self._IK_energy_fxn(δm_θh,δm_θhk,δm_θa)
        δpPE = self._IK_energy_fxn(δp_θh,δp_θhk,δp_θa)
        
        return -(δpPE - δmPE) / 2*δ
    
    def _IK_scipy_obj_fxn(self,ϕ,x,y):
        """defines an objective function for scipy.optimize.minimize"""
        θh,θk,θa = self._IK_analytical(x,y,ϕ)
        return self._IK_energy_fxn(θh,θk,θa)   
    
    def IK(self,x,y,method = "analytical"):
        """
        given a location x,y of the midfoot, what are the joint angles required to
        achieve that position? 
        """
        #check that IK can be performed
        #don't consider reaching just the foot
        if utils.dist(0,0,x,y) > self.femur + self.shank:
            return np.nan
        
        #method == analytical -> 3ms per evaluation...can we do better? 
        #determine ϕ of least potential energy
        if method == "analytical":
            ϕ_guess = self.θhSpring.θo + self.θkSpring.θo + self.θaSpring.θo
            res = minimize(self._IK_scipy_obj_fxn,(ϕ_guess),(x,y), method = 'nelder-mead')
            ϕ_min = res.x[0]
            θh,θk,θa = self._IK_analytical(x,y,ϕ_min)
            return θh,θk,θa
#             ϕ_min = utils.GD(ϕ_guess,self._energy_grad,(x,y))
#             θh,θk,θa = self._IK_analytical(x,y,ϕ_min)
#             return θh,θk,θa
        
        else:
            raise ValueError("method {} not yet implemented".format(method))

        #look into faster ways of optimizing this in the future - including a joint optimizatio
        #idea - define an energy function with springs, and an analytical 
            
        #method == numerical
            #idea - do the same thing as above, but solve the IK, then the energy function back
            #and forth
        #method == attractor use an attractor method with joint springs, like the one I used for 739 final project
        #which is solving the energy function at the same time as the 
        #method == optimization w/ bounds 
            #idea: solve with an explicit optimizer (cvxpy) - setup joint limits as constraints, 
        
        #method == fixed ankle
        
        #method = phi - field

        #return θh,θk,θa
    
    def _FK(self,θh,θk,θa):
        """
        calculate the forward kinematics of each joint and the contact point
        returns: 
            vectors of x and y coordinates of forward kinematic
            [hip,knee,ankle,midfoot,toe]
        """
        #calculate the sines and cosines of successive angles
        s_h   = np.sin(θh)        ; c_h   = np.cos(θh)
        s_hk  = np.sin(θh+θk)     ; c_hk  = np.cos(θh+θk)
        s_hka = np.sin(θh+θk+θa)  ; c_hka = np.cos(θh+θk+θa)
        
        #perform the forward kinematics 
        hip     = [0,0]
        knee    = [  hip[0] + self.femur*c_h ,      hip[1] + self.femur*s_h]                                     
        ankle   = [ knee[0] + self.shank*c_hk ,    knee[1] + self.shank*s_hk]
        midfoot = [ankle[0] +(self.foot/2)*c_hka, ankle[1] +(self.foot/2)*s_hka]
        toe     = [ankle[0] + self.foot*c_hka,    ankle[1] + self.foot*s_hka]
        
        #reorganize into vectors
        x = [hip[0],knee[0],ankle[0],midfoot[0],toe[0]]
        y = [hip[1],knee[1],ankle[1],midfoot[1],toe[1]]
        
        return (x,y)
 
    
    #torque calculation functions (from tables)
    def _τh(self,θh,θk):
        pass
        #develop some sort of spline interpolation fxn in n variables.
    
    def _τk(self,θh,θk,θa):
        pass
    
    def _τa(self,θk,θa):
        pass
    
    def _τspace(self):
        pass
        #τh = self.τh(θh,θk) ; τk = self.τh(θh,θk,θa) ; τa = self.τa(θk,θa)
    
    def _Jac(self,θh,θk,θa):
        pass
    
    def bounds(self, method = "full sample"):
        """
        return the bounds in the x,y plane over which the human leg can reach, given
        the set of joint limits
        """
        if method == "full sample" or method == "boundry sample":
            
            #calculate the set of points using sampling methods
            nh = 50; nk = 25; 
            pts = np.zeros((nh*nk + 1,2)); i = 0 #flat list
            inner = "hip"; #inner = "knee"
            
            #sample over the restricted space
            if inner == "knee":
                for θh in np.linspace(self.θh_min,self.θh_max,nh):
                    for θk in np.linspace(self.θk_min,self.θk_max,nk):
                        x,y = self._FK(θh,θk,0)
                        pts[i,:] = np.array((x[2],y[2]))
                        i += 1
            
            if inner == "hip":
                for θk in np.linspace(self.θk_min,self.θk_max,nh):
                    for θh in np.linspace(self.θh_min,self.θh_max,nk):
                        x,y = self._FK(θh,θk,0)
                        pts[i,:] = np.array((x[2],y[2]))
                        i += 1
                    
            #return sampled points
            if method == "full sample":
                return pts[:,0] , pts[:,1]
            
            #the workspace we want isn't convex and this proves it.
            if method == "boundry sample": 
                hull = ConvexHull(pts)
                hullPts = pts[hull.vertices,:]
                return hullPts[:,0] , hullPts[:,1]
        
        
        #plot the outer reach of the leg,without regard to joint limits
        if method == "circle":
            t = np.linspace(0,2* np.pi,100)
            r = self.reach
            x = r * np.cos(t)
            y = r * np.sin(t)
            return x,y

    
    # ---------------------- public facing functions ------------------------------
    def FK(self,θh,θk,θa):
        """
        given the joint angles θh,θk,θa, where is the middle of the foot located in space?
        """
        x,y = self._FK(θh,θk,θa) ; return (x[3],y[3])
    
    def FKplot(self,θh,θk,θa):
        """
        forward kinematic plotting vector for viewing human leg
        """
        return self._FK(θh,θk,θa)
        
    def FS(self,x,y):
        """
        evaluate the Force Space of the human at a particular location location in the saggital plane
        and return a shape object that can be used to score FS overlap
        """
        self.IK(x,y)  #update internal state
        jac = self.Jac()
        τs = self.τspace()
        
        #jac * τs -> force space
        return fs
    
    def FSplot(self,x,y):
        """
        return the points required to plot the force space
        """
        pass
    
    def IKplot(self,x,y):
        """return the points for plotting human leg"""
        pass
        
        
         
    

#----------------------------------- tests ----------------------------------
def test_human():
    h = Human()
    def test_FS():
        pass
    pass

test = False
if test:
    test_human()
    
        
#---------------------------------- plotting ---------------------------------
h = Human()
hfig = go.FigureWidget()

#configure layout of graph
hfig.update_xaxes(range=[-1.2*h.reach, 1.2*h.reach])
hfig.update_yaxes(range=[-1.2*h.reach, 1.2*h.reach])

side = 1000
hfig.update_layout(
    autosize=False,
    width=side + 100,
    height=side,
    legend=dict(x=.025, y=.975),
    margin=dict(l=80, r=80, t=0, b=80))


#initialize all plots present in visualization
hfig.add_scatter(line=dict(dash='dash',color='royalblue'), name = "workspace")
hfig.add_scatter(mode="markers", marker = dict(color = np.linspace(0,1,50*25)),name="boundrySample")
hfig.add_scatter(line=dict(color='royalblue'),name = "leg")
hfig.add_scatter(mode="markers", name = "forceSpace")


#setup UI
rng = h.reach; step = .1
layout=Layout(width='500px', height='20px')
layout1=Layout(width='350px', height='30px')

workspace = Checkbox(value=True,description="workspace",dindent=False)
boundsSample = Checkbox(value=False,description="boundsSample",dindent=False)
forceSpace = Checkbox(value=False,description="forceSpace",dindent=False)

fsScale = FloatSlider(min=.25,max=3,step=step,value=1,description='Force Scale',layout=layout)
x1 = FloatSlider(min=-rng,max=rng,step=step,value=.75,description='x', layout=layout)
y1 = FloatSlider(min=-rng,max=rng,step=step,value=0,description='y', layout=layout)

θh_0 = FloatSlider(min=-np.pi,max=np.pi,step=step,value=h.θhSpring.θo,description='θh_0', layout=layout1)
θk_0 = FloatSlider(min=-np.pi,max=np.pi,step=step,value=h.θkSpring.θo,description='θk_0', layout=layout1)
θa_0 = FloatSlider(min=-np.pi,max=np.pi,step=step,value=h.θaSpring.θo,description='θa_0', layout=layout1)

θh_k = FloatSlider(min=.1,max=5,step=step,value=h.θhSpring.K,description='θh_k', layout=layout1)
θk_k = FloatSlider(min=.1,max=5,step=step,value=h.θkSpring.K,description='θk_k', layout=layout1)
θa_k = FloatSlider(min=.1,max=5,step=step,value=h.θaSpring.K,description='θa_k', layout=layout1)


#assemble UI
row1 = widgets.HBox([workspace,boundsSample,forceSpace])
row2 = widgets.HBox([x1,fsScale])
row3 = widgets.HBox([y1])
row4 = widgets.HBox([θh_0,θk_0,θa_0])
row5 = widgets.HBox([θh_k,θk_k,θa_k])

ui = widgets.VBox([row1,row2,row3,row4,row5])

wdict = {"workspace":workspace,"boundsSample":boundsSample,"forceSpace":forceSpace,
         "x1":x1,"fsScale":fsScale,
         "y1":y1,
         "θh_0":θh_0,"θk_0":θk_0,"θa_0":θa_0,
         "θh_k":θh_k,"θk_k":θk_k,"θa_k":θa_k}

def update(workspace=True,boundsSample=False,forceSpace=False,
           x1=1.5,fsScale=1,
           y1=0,
           θh_0=0,θk_0=0,θa_0=0,
           θh_k=0,θk_k=0,θa_k=0):
    
    with hfig.batch_update():
        #update the spring constants within the human
        h.θhSpring.θo = θh_0 
        h.θkSpring.θo = θk_0
        h.θaSpring.θo = θa_0
        
        h.θhSpring.K = θh_k
        h.θkSpring.K = θk_k
        h.θaSpring.K = θa_k
        
        #draw the workspace (eventually, just the boundry once that fxn is done)
        if workspace:
            xs,ys = h.bounds(method="circle")
            hfig.data[0]['x'] = xs
            hfig.data[0]['y'] = ys
        else:
            hfig.data[0]['x'] = []
            hfig.data[0]['y'] = []
            
        #draw bounds Sample
        if boundsSample:
            xs,ys = h.bounds()
            hfig.data[1]['x'] = xs
            hfig.data[1]['y'] = ys
        else:
            hfig.data[1]['x'] = []
            hfig.data[1]['y'] = []
            
        
        #draw the human leg
        θh,θk,θa = h.IK(x1,y1,method = "analytical")
        xs,ys = h.FKplot(θh,θk,θa)
        #xs,ys = h.FKplot(0,0,0)
        hfig.data[2]['x'] = xs
        hfig.data[2]['y'] = ys
    
    
        #draw the force space
        #tbd.


#display interactive figure
out = widgets.interactive_output(update, wdict)
display(ui)
hfig

FigureWidget({
    'data': [{'line': {'color': 'royalblue', 'dash': 'dash'},
              'name': 'workspace'…

## discussion points
* need to invest more time into developing the IK, current optimization method is too slow. (300Hz)

In [7]:
xs,ys = h.FKplot(-np.pi/4,-.5,np.pi/4)
hfig.data[2]['x'] = xs
hfig.data[2]['y'] = ys

to do: what are my goals for this visualizer?
* be able to debug the human class
* visualize the bounds and bounds check function
* visualize the process of IK, and how it's being solved
* visualize the force space of the human


what am I trying to do right now?
- make the buttons and plots for the human visualizer

the calculated space depends on whether the robot's are elbow in or elbow out.
$det(A^{-1}) = \frac{1}{det(A)}$

in words, how do you want this to work? 
* when visualize forceSpaces is selected, you have free control over the movement of \theta1 and \theta2, and can see the calculated force spaces. FR scales the robot force space. when you select that button, the blue robot becomes bold, and is the leader, the robot follows through IK. if the human leaves the area, then robot should disappear, until we are back in the interaction area. 
* the feature is separate from and is drawn over top of the mode controur map. 
    
* what am I trying to do right now? 
- build the visualization tool that will help explain why the inscribed and actuator effort modes have such asymmetries 

<a id="joint_exp"></a>
## Joint exploration - put the two together [ &#x21ea;](#top)


<a id="solution"></a>
## Solve the constrained optimization problem [ &#x21ea;](#top)

<a id="notes"></a>
## notes:

speed up code with cython
https://jakevdp.github.io/blog/2017/12/11/live-coding-cython-ising-model/?fbclid=IwAR1_KPRJACFUJk6-TYb1DqumE9Cn3hk40QUOl10UCyItL2z4Ft5x_s-Nc60
numba, JIT

python optimization with pyomo http://egon.cheme.cmu.edu/ewo/docs/EWO_Seminar_03_10_2017.pdf
